In [19]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import sys
sys.tracebacklimit = 0

import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt

In [21]:
from pettingzoo.test import parallel_api_test
from solution.custom_gym import CustomGymEnviornment
from solution.trainer import *
from solution.policy_net import *

In [22]:
from core.agent import *
from core.world import * 
from core.render import * 
from core.skill import * 
from core.models import *
from core.message import *

In [23]:
DEVICE = "cuda"

In [24]:
from sar.sar_agent import *
from sar.sar_world import *
from sar.sar_env_params import *
from sar.sar_traits_sampler import *

belief_initializer = SARBeliefInitializer(BELIEF_DIMS)
trait_sampler = SARTraitSampler()

def initialize_swarm(world : BaseWorld):
    swarm = trait_sampler.generate(SWARM_SIZE, DEVICE)
    for agent in swarm:
        agent.set_utility(SARUtilityFunction())
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    swarm = belief_initializer.initialize_beliefs(swarm)

In [25]:

from sar.urban_gen import * 
from sar.victims import * 
from sar.sar_comm import * 

terrain_generator = UrbanTerrainMapGenerator(padding = MAX_VISIBILITY)
victim_generator = VictimGenerator(padding = MAX_VISIBILITY)
def initialize_terrain(world : BaseWorld):
    terrain_map, population_map = terrain_generator.generate(world._dims)
    map_collection : BaseMapCollection = BaseMapCollection()
    map_collection.add_map("Terrain", terrain_map)
    map_collection.add_map("Population", population_map)

    victim_generator.set_density_map(population_map)
    victim_map = victim_generator.generate(world._dims)

    map_collection.add_map("Victims", victim_map)
    return map_collection


In [26]:
from sar.energy import EnergyModel
from sar.victims import VictimModel
from solution.sar_action_interpreter import *
from solution.encoder_net import *
from solution.decoder_net import *
from models.complex_model import * 

world = SARWorld(dims = WORLD_DIMS,
              swarm_initializer= initialize_swarm,
              generation_pipeline=initialize_terrain
              )
world.add_model("energy_model", EnergyModel())
world.add_model("victim_model", VictimModel())
world.reset()

In [27]:
policy_net= PolicyNet(1, 7, 4)
target_net=  PolicyNet(1, 7, 4)
encoder_model = Encoder()
decoder_model = Decoder()

complex_model = ComplexModel(
    policy_net= policy_net, 
    encoder_net = encoder_model, 
    decoder_net = decoder_model
)

comms_protocol = SARCommunicationProtocol(encoder_model, decoder_model)
action_interpreter = SARActionInterpreter(BELIEF_DIMS)

custom_gym : CustomGymEnviornment = CustomGymEnviornment(world, action_interpreter, comms_protocol)

complex_model.to(DEVICE)
custom_gym.to(DEVICE)

In [28]:
custom_gym.reset(42)

({1: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 0., 0., 1., 1., 0., 1.],
          [1., 0., 0., 1., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 1., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0., 1., 0.],
          [1., 0., 0., 1., 0., 0., 0.]]),
   'State': tensor([90.3885,  1.0000], device='cuda:0')},
  2: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 1., 0., 0., 0.],
          [0., 0., 1., 1., 1., 1., 0.],
          [0., 0., 1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 1., 0., 0.],
          [1., 1., 0., 1., 1., 1., 1.],
          [1., 0., 1., 1., 0., 0., 1.],
          [1., 1., 0., 0., 0., 0., 0.]]),
   'State': tensor([107.1394,   0.0000], device='cuda:0')},
  3: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 0., 0., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1.],
          [1., 0., 

### Testing

In [29]:
parallel_api_test(custom_gym, num_cycles=1_000)
custom_gym.reset()

Passed Parallel API test


({1: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[0., 0., 0., 1., 1., 0., 1.],
          [1., 0., 0., 1., 0., 0., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 1., 0.],
          [1., 1., 0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0., 1., 0.],
          [1., 0., 0., 1., 0., 0., 0.]]),
   'State': tensor([90.3885,  1.0000], device='cuda:0')},
  2: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 1., 0., 0., 0.],
          [0., 0., 1., 1., 1., 1., 0.],
          [0., 0., 1., 0., 0., 0., 0.],
          [1., 0., 0., 0., 1., 0., 0.],
          [1., 1., 0., 1., 1., 1., 1.],
          [1., 0., 1., 1., 0., 0., 1.],
          [1., 1., 0., 0., 0., 0., 0.]]),
   'State': tensor([107.1394,   0.0000], device='cuda:0')},
  3: {'Belief': tensor([0., 0., 0., 0., 0.], device='cuda:0'),
   'Vision': array([[1., 1., 1., 0., 0., 1., 1.],
          [0., 0., 1., 1., 1., 1., 1.],
          [1., 0., 

# Training

In [30]:

from models.base import * 
from models.idqn import * 
from solution.policy_net import PolicyNet
import matplotlib.pyplot as plt

In [31]:
model = IDQN(env = custom_gym,
             feature_extractor= feature_extractor,
             target_net= target_net,
             model= complex_model,
             batch_size=1024,
             device = DEVICE
             )

In [ ]:
rewards = train_loop(custom_gym, model, games=10, optimization_passes = 1, seed=42)

In [ ]:
plt.plot(rewards)

# Optimizations

In [32]:
import cProfile

In [33]:
def stress_test():
    train_loop(custom_gym, model, games=1, optimization_passes = 1, seed=42)

In [35]:

cProfile.run('stress_test()', sort = 'time')

Training on thesis.


Training Progress:   0%|          | 0/1 [00:00<?, ?it/s]

Average loss 9.337492402747273
Model has been saved.

Starting evaluation on thesis (num_games=1)


Training Progress: 100%|██████████| 1/1 [05:03<00:00, 303.39s/it]

Avg reward: 333.045  std: 299.59614980002664  coeff : 0.8995665744870112
Avg reward per agent, per game:  {1: 584.0, 2: 197.0, 3: 0.0, 4: 99.0, 5: 1028.0, 6: 561.0, 7: 546.0, 8: 196.0, 9: 485.0, 10: 387.0, 11: 0.0, 12: 480.0, 13: 385.0, 14: 1253.0, 15: 463.0, 16: 369.0, 17: 294.0, 18: 281.0, 19: 99.0, 20: 584.0, 21: 487.0, 22: 1094.0, 23: 99.0, 24: 566.0, 25: 468.0, 26: 198.0, 27: 198.0, 28: 382.0, 29: 284.0, 30: 564.0, 31: 294.0, 32: 295.0, 33: 474.0, 34: 190.0, 35: 0.0, 36: 0.0, 37: 582.0, 38: 365.0, 39: 0.0, 40: 1446.0, 41: 382.0, 42: 390.0, 43: 294.0, 44: 450.0, 45: 849.0, 46: 655.0, 47: 99.0, 48: 763.0, 49: 99.0, 50: 99.0, 51: 294.0, 52: 0.0, 53: 393.0, 54: 198.0, 55: 390.0, 56: 387.0, 57: 0.0, 58: 728.0, 59: 99.0, 60: 99.0, 61: 88.0, 62: 0.0, 63: 99.0, 64: 761.0, 65: 198.0, 66: 670.0, 67: 0.0, 68: 192.0, 69: 197.0, 70: 195.0, 71: 674.0, 72: 0.0, 73: 294.0, 74: 99.0, 75: 473.0, 76: 812.0, 77: 195.0, 78: 0.0, 79: 278.0, 80: 198.0, 81: 0.0, 82: 296.0, 83: 290.0, 84: 99.0, 85: 388.0,